In [4]:
import pandas as pd

In [5]:
lexique_df = pd.read_csv("../paraphone/paraphone/data/dictionaries/lexique_383.tsv", delimiter="\t")

In [6]:
short_df = lexique_df[["ortho", "phon", "syll"]]

In [9]:
phonemes = set()
for i, row in short_df.iterrows():
    phonemes.update(set(row["phon"]))
phonemes

{'1',
 '2',
 '5',
 '8',
 '9',
 '@',
 'E',
 'G',
 'N',
 'O',
 'R',
 'S',
 'Z',
 'a',
 'b',
 'd',
 'e',
 'f',
 'g',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '§',
 '°'}

In [10]:
len(lexique_df)

142694

In [13]:
import  re
regex = re.compile("8(.)")
with_8 = set()
for i, row in short_df.iterrows():
    phones: str = row["phon"]
    with_8.update(set(regex.findall(phones)))
with_8

{'2', '5', '9', '@', 'E', 'O', 'a', 'e', 'i', 'o', 'y', '§'}

In [ ]:
# Investigations for the CELEX dict

In [30]:
import csv
all_phonemes_chars = set()
all_phonemes = set()
with open("../paraphone/data/foldings/en/celex.csv") as celex_fold_file:
    reader = csv.DictReader(celex_fold_file, delimiter=",")
    for row in reader:
        phon_celex = row["phon_celex"]
        all_phonemes_chars.update(set(phon_celex))
        all_phonemes.update(phon_celex.split(" "))
all_phonemes_chars.remove(" ")

In [35]:
all_phonemes_chars

{'3',
 ':',
 '@',
 'A',
 'D',
 'E',
 'I',
 'N',
 'O',
 'Q',
 'S',
 'T',
 'U',
 'V',
 'Z',
 'a',
 'b',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'z',
 '{',
 '~'}

In [34]:
from tqdm import tqdm
import re
clx_phon_re = re.compile(r"\[(.+)\]")

rejected_phonemes_char = set()
total_count, rejected_count = 0, 0
with open("../data/epw.cd") as celex_dic, open("../data/rejected.csv", "w") as rej_file:
    rej_csv = csv.writer(rej_file, delimiter="\t")
    for row in tqdm(celex_dic):
        row = row.strip().split("\\")
        total_count += 1
        re_match = clx_phon_re.findall(row[6])
        phonemized_word = "".join(re_match)
        rejected = set(phonemized_word) - all_phonemes_chars
        if rejected:
            rejected_count += 1
            rejected_phonemes_char.update(rejected)
            rej_csv.writerow((row[1], row[6], rejected))

total_count, rejected_count

160595it [00:00, 450926.01it/s]


(160595, 0)

In [20]:
set(phonemized_word)

{'1'}